# Setup

In [1]:
import os
import pandas as pd
import json

In [2]:
# Định nghĩa các đường dẫn cần thiết
input_folder = r'D:\HTP\Project-personal\CoinAnalytics\Main\data\processed'
output_folder = r'D:\HTP\Project-personal\CoinAnalytics\Main\data\warehouse'

# Danh sách các file CSV và tên các token
csv_files = [
    "ALICEUSDT_processed.csv", "AXSUSDT_processed.csv", "ENJUSDT_processed.csv", 
    "GALAUSDT_processed.csv", "MANAUSDT_processed.csv", "SANDUSDT_processed.csv", 
    "SUSHIUSDT_processed.csv", "YGGUSDT_processed.csv"
]
token_names = ['ALICE', 'AXS', 'ENJ', 'GALA', 'MANA', 'SAND', 'SUSHI', 'YGG']

# Kiểm tra và tạo thư mục lưu kết quả nếu chưa tồn tại
os.makedirs(output_folder, exist_ok=True)

In [3]:
# Đọc các file CSV và lưu vào dictionary
token_prices_dfs = {}
for file in csv_files:
    token_name = file.split('USDT')[0]  # Lấy tên token từ tên file
    file_path = os.path.join(input_folder, file)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        token_prices_dfs[token_name] = df
    else:
        print(f"File {file} không tồn tại tại {file_path}")

# Đọc file JSON chứa thông tin token
token_info_file = os.path.join(input_folder, "token_info.json")
if os.path.exists(token_info_file):
    with open(token_info_file, "r") as f:
        token_info_data = json.load(f)
else:
    raise FileNotFoundError(f"File {token_info_file} không tồn tại.")

# dim_token.csv

In [4]:
tokens_df = pd.DataFrame({
    "token_id": range(1, len(token_names) + 1),
    "token_name": [desc.get("Description", "").split(" (")[0] for desc in token_info_data],
    "token": token_names
})

# dim_token_info.csv

In [5]:
token_info_df = pd.DataFrame({
    "token_id": range(1, len(token_info_data) + 1),
    "description": [token.get("Description", "") for token in token_info_data],
    "website": [token.get("Website", "") for token in token_info_data],
    "technical_doc": [token.get("Technical Doc", "") for token in token_info_data],
    "twitter": [token.get("Twitter", "") for token in token_info_data],
    "reddit": [token.get("Reddit", "") for token in token_info_data],
    "logo": [token.get("Logo", "") for token in token_info_data]
})

# fact_token_prices.csv

In [6]:
# Gộp dữ liệu giá token và thêm cột token_id
token_prices_df = pd.DataFrame()
for token_name, df in token_prices_dfs.items():
    token_id = token_names.index(token_name) + 1
    df['token_id'] = token_id  # Thêm token_id vào DataFrame
    token_prices_df = pd.concat([token_prices_df, df], ignore_index=True)

# Đưa token_id lên đầu trong bảng token_prices_df
columns_order = ['token_id'] + [col for col in token_prices_df.columns if col != 'token_id']
token_prices_df = token_prices_df[columns_order]

# Save

In [7]:
# Lưu các bảng dưới dạng CSV
tokens_df.to_csv(os.path.join(output_folder, 'dim_token.csv'), index=False, encoding='utf-8-sig')
token_info_df.to_csv(os.path.join(output_folder, 'dim_token_info.csv'), index=False, encoding='utf-8-sig')
token_prices_df.to_csv(os.path.join(output_folder, 'fact_token_prices.csv'), index=False, encoding='utf-8-sig')

print("Dữ liệu đã được chia và lưu vào thư mục warehouse.")

Dữ liệu đã được chia và lưu vào thư mục warehouse.
